# **R-Squared Disentangled**

### Import Libraries

In [34]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

pd.set_option('display.max_columns', 500)

### Import Data

In [35]:
dummy = pd.read_excel('../../Dados/crypto_fear_greed_data.xlsx')
dummy

,date,daily_index,sentiment,weekly_mean,montly_mean
0,2023-10-23,53,Neutral,NaN,NaN
1,2023-10-22,53,Neutral,52.857143,NaN
2,2023-10-21,63,Greed,NaN,NaN
3,2023-10-20,53,Neutral,NaN,NaN
4,2023-10-19,52,Neutral,NaN,NaN
...,...,...,...,...,...
2083,2018-02-05,11,Extreme Fear,NaN,NaN
2084,2018-02-04,24,Extreme Fear,27.250000,NaN
2085,2018-02-03,40,Fear,NaN,NaN
2086,2018-02-02,15,Extreme Fear,NaN,NaN


## *R-Squared*

In [36]:
fg = dummy.drop(['sentiment', 'weekly_mean', 'montly_mean'], axis=1).set_index('date').sort_values(by='date', ascending=True)
fg['weekly_mean'] = fg['daily_index'].rolling(7).mean()
fg['sentiment'] = 0

#fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
#fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
#fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
#fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
#fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg

,daily_index,weekly_mean,sentiment
date,,,
2018-02-01,30,NaN,0
2018-02-02,15,NaN,0
2018-02-03,40,NaN,0
2018-02-04,24,NaN,0
2018-02-05,11,NaN,0
...,...,...,...
2023-10-19,52,48.142857,Neutral
2023-10-20,53,49.428571,Neutral
2023-10-21,63,51.714286,Neutral


In [37]:
tech = pd.read_excel('../../Dados/tech_signals_w.xlsx')
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [38]:
df = tech.merge(fg, how='left', left_on='Date', right_on='date')
df.dropna(inplace=True)
df.drop('daily_index', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-12,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.000000,Fear
155,2018-02-19,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,Greed
156,2018-02-26,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,Neutral
157,2018-03-05,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,Neutral
158,2018-03-12,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2023-09-25,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,Neutral
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,Neutral
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,Neutral
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,Neutral


In [39]:
rf = pd.read_excel('../../Dados/risk_free_d.xlsx')
rf['r_free_day'] = rf['r_free_day'].ffill()
rf['r_free_w'] = rf['r_free_day'].rolling(7).mean()
rf = rf[['observation_date', 'r_free_w']]
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.head(10)

,observation_date,r_free_w
0,2018-11-06,NaN
1,2018-11-07,NaN
2,2018-11-08,NaN
3,2018-11-09,NaN
4,2018-11-12,NaN
5,2018-11-13,NaN
6,2018-11-14,0.000087
7,2018-11-15,0.000087
8,2018-11-16,0.000087
9,2018-11-19,0.000087


In [40]:
dummies = pd.get_dummies(df['sentiment'])
df = pd.concat([df.drop('sentiment', axis=1), dummies], axis=1)
df['Any'] = [True]*len(df)
df['Returns'] = df['Adj Close'].pct_change()
df = df.merge(rf, how='left', left_on='Date', right_on='observation_date').dropna()
df.drop('observation_date', axis=1, inplace=True)
df['risk_premium'] = df['Returns'] - df['r_free_w']
df.head(10)

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,Fear,Greed,Neutral,Any,Returns,r_free_w,risk_premium
39,2018-11-19,4009.969971,42533970000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.428571,True,False,False,True,-0.286931,0.000087,-0.287019
40,2018-11-26,4139.877930,42945276472,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.714286,True,False,False,True,0.032396,0.000088,0.032308
41,2018-12-03,3614.234375,38386467708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.285714,True,False,False,True,-0.126971,0.000090,-0.127061
42,2018-12-10,3252.839111,29869247431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.142857,True,False,False,True,-0.099992,0.000092,-0.100084
43,2018-12-17,3998.980225,46021506218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13.428571,True,False,False,True,0.229381,0.000091,0.229290
44,2018-12-24,3865.952637,39249734918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.142857,True,False,False,True,-0.033265,0.000093,-0.033359
45,2018-12-31,4076.632568,34343716582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.000000,True,False,False,True,0.054496,0.000095,0.054401
46,2019-01-07,3552.953125,37523454177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.428571,True,False,False,True,-0.128459,0.000095,-0.128554
47,2019-01-14,3601.013672,38588596987,0,0,0,0,0,0,0,0,1,0,0,0,0,0,28.000000,True,False,False,True,0.013527,0.000095,0.013432
48,2019-01-21,3583.965820,36949379050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.142857,True,False,False,True,-0.004734,0.000094,-0.004829


In [41]:
r2 = {}

for i in range(3, 17):
    y = df['risk_premium'][1:df.shape[0]].values
    x = df.iloc[0:df.shape[0]-1, i].values
    x = sm.add_constant(x)  
    model = sm.OLS(y, x).fit()
    e = model.resid
    Ic = []
    for j in range(18, 22):
        r_squared = 1 - (np.sum(df.iloc[1:df.shape[0], j] * e**2) / np.sum(df.iloc[1:df.shape[0], j] * ((y - np.mean(y))**2)))
        Ic.append(r_squared)
    r2[df.columns[i]] = Ic

In [42]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T 
r2_df.columns = ['Fear', 'Greed', 'Neutral', 'Any']
r2_df

,Fear,Greed,Neutral,Any
MA_1_9,0.023836,0.016333,0.006353,0.018789
MA_1_12,0.007452,0.013515,-0.009334,0.007034
MA_2_9,0.006030,0.013413,0.009570,0.009009
MA_2_12,0.007060,0.012416,-0.004587,0.007153
MA_3_9,-0.001969,0.025576,0.021953,0.010704
MA_3_12,-0.003651,0.021567,0.009103,0.006621
MOM_9,0.010483,0.041135,0.001307,0.019378
MOM_12,0.001676,0.009238,-0.008681,0.002693
VOL_1_9,0.005682,0.021306,0.036383,0.015362
VOL_1_12,0.002814,0.011719,0.018067,0.008005


In [43]:
import plotly.express as px

In [44]:
r2_df.index = ['MA(1,9)','MA(1,12)','MA(2,9)','MA(2,12)','MA(3,9)','MA(3,12)','MOM(9)','MOM(12)','VOL(1,9)','VOL(1,12)','VOL(2,9)','VOL(2,12)','VOL(3,9)','VOL(3,12)']

cores2 = [px.colors.qualitative.D3[3], px.colors.qualitative.T10[5], px.colors.qualitative.G10[7]]

fig = px.bar(r2_df*100, r2_df.index, ["Fear", "Neutral", "Greed"], 
             title="Figura 1. In-sample R² disentangled.", color_discrete_sequence=cores2,
             labels={"value": "R² (%)", "index": "", "variable": ""})
fig.add_hline(y=0.5, line_width=1.2, line_dash="dash", line_color="red")
fig.update_layout(barmode='group',bargroupgap=0, bargap=0.4, width=1000, height=400, title_x=0.08, title_y=0.03)
fig.update_xaxes(tickangle=25)
fig.show()